In [2]:
using Pkg
Pkg.activate(".")


  Activating project at `~/Documents/GaussianProcessNode`


In [3]:
using ReactiveMP, GraphPPL,RxInfer, Revise, KernelFunctions, LinearAlgebra

[ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

[ Info: Precompiling RxInfer [86711068-29c9-4ff7-b620-ae75d7495b3d]
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [22]:
xmin, xmax = -3, 3  # Bounds of the data
N = 20# Number of samples
xtrain = sort(rand(Uniform(xmin, xmax), N))
xtest = collect(range(xmin - 0.1, xmax + 0.1; length=60));
# xtest = sort(vcat(xtest,xtrain))
precision = 100
ytrain = sinc.(xtrain) + randn(N) * sqrt(1/precision);
ytrain[10:11] =  ytrain[10:11] .+ 2
f_true = sinc.(xtest);

In [23]:
@model function tprocess(n,kernelfunc,meanfunc,train,test)
    y = datavar(Float64,n)
    x = randomprocess(test,train, ReactiveMP.CovarianceMatrixStrategy(FullCovarianceStrategy()))
    x ~ GeneralizedTProcess(meanfunc,kernelfunc,100,log(2.0)) 
    for i=1:n 
        y[i] ~ NormalMeanPrecision(x, 100.0) where {meta=ProcessMeta(i)}
    end
end

In [27]:
@model function gprocess(n,kernelfunc,meanfunc,train,test)
    y = datavar(Float64,n)
    x = randomprocess(test,train, ReactiveMP.CovarianceMatrixStrategy(FullCovarianceStrategy()))
    x ~ GaussianProcess(meanfunc,kernelfunc,log(2.0)) 
    for i=1:n 
        y[i] ~ NormalMeanPrecision(x, 100.0) where {meta=ProcessMeta(i)}
    end
end

In [7]:
# @constraints function gpconstraints()   
#     q(x,γ) = q(x)q(γ)
# end

k_func(θ) =  with_lengthscale(Matern52Kernel(),θ)
mean_func   = (x) -> 0.0;

In [24]:
iresult_tp = inference(
    model = tprocess(length(ytrain), k_func, mean_func, xtrain,xtest),
    data  = (y = ytrain,),
    free_energy = false,
)


Inference results:
  Posteriors       | available for (x)


In [42]:
mx_tp = mean(iresult_tp.posteriors[:x].data.finitemarginal)
vx_tp = var(iresult_tp.posteriors[:x].data.finitemarginal);
samples_tp = rand(iresult_tp.posteriors[:x].data.finitemarginal,100);

In [28]:
iresult_gp = inference(
    model = gprocess(length(ytrain), k_func, mean_func, xtrain,xtest),
    data  = (y = ytrain,),
    free_energy = false,
)


Inference results:
  Posteriors       | available for (x)


In [40]:
mx_gp = mean(iresult_gp.posteriors[:x].data.finitemarginal)
vx_gp = var(iresult_gp.posteriors[:x].data.finitemarginal);
samples_gp = rand(iresult_gp.posteriors[:x].data.finitemarginal,100);

In [52]:
using Plots, LaTeXStrings
pgfplotsx()
plot(xtest,mx_tp,ribbon=sqrt.(vx_tp),color=:blue, lw=2.0,size=(800,400),fillalpha=0.2,label="t-process")
plot!(xtest,mx_gp,ribbon=sqrt.(vx_gp),color=:green, lw=2.0,fillalpha=0.2,legend=:topleft,label="gaussian process")
scatter!(xtrain,ytrain,color=:black,ms=1.0,label="observations")
plot!(xtest, f_true, color=:black, lw=2.0,label="true function")
xlabel!(L"x")
ylabel!(L"f(x)")
savefig("regresssion.svg")

"/Users/isenoz/Documents/GaussianProcessNode/regresssion.svg"

In [53]:
plot(samples_tp,color=:blue,alpha=0.2,label=false,size=(800,400))
plot!(samples_gp,color=:green,alpha=0.2,label=false,size=(800,400))
xlabel!(L"x")
ylabel!(L"f(x)")
savefig("samples.svg")

"/Users/isenoz/Documents/GaussianProcessNode/samples.svg"